            # PhysAE – inférence et génération de soumissions

            Ce carnet démontre comment charger un checkpoint, produire des
            prédictions et créer des fichiers de soumission aux formats CSV et
            JSON.
            


            ## 1. Chargement du modèle et des données de test

            Nous supposons que `stage_B2.ckpt` est disponible (généré via le carnet
            de démarrage rapide ou fourni par l'équipe).
            


In [ ]:
            from pathlib import Path
            import json
            import pandas as pd
            import torch

            from physae.factory import build_data_and_model

            ARTIFACT_DIR = Path('artifacts/quickstart')
            CKPT_PATH = ARTIFACT_DIR / 'stage_B2.ckpt'
            assert CKPT_PATH.exists(), 'Veuillez exécuter le carnet 01 pour générer un checkpoint.'

            model = torch.load(CKPT_PATH, map_location='cpu') if CKPT_PATH.suffix == '.pt' else None
            if model is None:
                from physae.model import PhysicallyInformedAE
                model = PhysicallyInformedAE.load_from_checkpoint(str(CKPT_PATH))
            model.eval()
            if torch.cuda.is_available():
                model.cuda()

            _, test_loader, _ = build_data_and_model(config_overrides={'n_train': 0, 'n_val': 0, 'batch_size': 16})
            print('Nombre de lots de test :', len(test_loader))
            


            ## 2. Boucle d'inférence

            On applique la méthode `infer` du modèle pour chaque lot et on
            dénormalise les paramètres physiques avant de les stocker.
            


In [ ]:
            from physae.normalization import unnorm_param_torch

            predictions = []
            with torch.no_grad():
                for batch in test_loader:
                    noisy = batch['noisy_spectra'].to(model.device)
                    provided = {
                        name: batch['params'][:, model.name_to_idx[name]].to(model.device)
                        for name in model.provided_params
                    }
                    outputs = model.infer(noisy, provided, refine=True)
                    phys = outputs['y_phys_full'].cpu()
                    for i, sample_id in enumerate(batch['id']):
                        row = {'sample_id': sample_id}
                        for name in model.predict_params:
                            idx = model.name_to_idx[name]
                            row[name] = float(unnorm_param_torch(name, phys[i, idx]))
                        predictions.append(row)
            print('Prédictions collectées :', len(predictions))
            


            ## 3. Création des fichiers de soumission

            Nous générons deux formats : CSV complet et JSON partiel.
            


In [ ]:
            csv_path = Path('docs/examples/submissions/submission_nominal.csv')
            pd.DataFrame(predictions).to_csv(csv_path, index=False)
            print('CSV écrit dans', csv_path)

            json_path = Path('docs/examples/submissions/submission_partial.json')
            json_payload = [
                {
                    'sample_id': row['sample_id'],
                    'params': {k: row[k] for k in model.predict_params[:3]},
                    'metadata': {'model': 'PhysAE', 'refine_steps': model.refine_steps},
                }
                for row in predictions
            ]
            json_path.write_text(json.dumps(json_payload, indent=2))
            print('JSON écrit dans', json_path)
            
